Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

Load model

In [ ]:
model = cobra.io.read_sbml_model("./models/yeast-GEM-BiGG.xml")
model

Objective function is `GROWTH` in the yeast-GEM model, inspecting it and related reactions:

In [ ]:
model.reactions.get_by_id('GROWTH')

In [ ]:
model.metabolites.get_by_id('biomass_c')

In [ ]:
# This is the biomass pseudoreaction -- we're interested in the stoichiometry of this.
biomass = model.reactions.get_by_id('BIOMASS_yeast-GEM')
biomass
# Conveniently, it has one 'metabolite' for all lipids, one for all proteins,
# one for RNA, etc.  We want to remove each in turn, so if it's written in this way,
# it's super easy to do so.

Inspecting each type of macromolecule...

Lipid

In [ ]:
model.metabolites.get_by_id('s_1096')

In [ ]:
# Lipid pseudoreaction is combining 'bulk' lipid backbone and 'bulk' lipid chain substrates
model.reactions.get_by_id('BIOMASS_yeast-GEM_LIP')

Protein

In [ ]:
model.metabolites.get_by_id('protein_c')

In [ ]:
# Protein pseudoreaction is combining aminoacyl-tRNAs
model.reactions.get_by_id('BIOMASS_yeast-GEM_PROT')

DNA

In [ ]:
model.metabolites.get_by_id('dna_c')

In [ ]:
# DNA pseudoreaction is combining dNTPs
model.reactions.get_by_id('BIOMASS_yeast-GEM_DNA')

RNA

In [ ]:
model.metabolites.get_by_id('rna_c')

In [ ]:
# RNA pseudoreaction is combining NTPs
model.reactions.get_by_id('BIOMASS_yeast-GEM_RNA')

Simulate model and draw fluxes through central carbon metabolism

In [ ]:
solution = model.optimize()
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b

Limit glucose uptake (minimal media)

In [ ]:
model.reactions.EX_glc__D_e.bounds = (-18.5, 1) # TODO, find a sensible number

Modify biomass reaction

Option A: Remove lipid

In [ ]:
biomass.subtract_metabolites({
    model.metabolites.get_by_id('s_1096'): -1
})
solution = model.optimize()

Option B: Remove protein

In [ ]:
biomass.subtract_metabolites({
    model.metabolites.get_by_id('protein_c'): -1
})
solution = model.optimize()

Option C: Remove carbohydrate

In [ ]:
biomass.subtract_metabolites({
    model.metabolites.get_by_id('s_3718'): -1
})
solution = model.optimize()

Option D: Remove DNA

In [ ]:
biomass.subtract_metabolites({
    model.metabolites.get_by_id('dna_c'): -1
})
solution = model.optimize()

Draw fluxes through central carbon metabolism

In [ ]:
biomass

In [ ]:
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b

Estimating time scale based on flux

In [ ]:
model.summary()